## Sentiment Analysis

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import s3fs
import spacy

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from ast import literal_eval
import re

from textblob import TextBlob

In [18]:
pd.set_option('display.max_colwidth', -1)

In [3]:
bucket = 's3://cortex-ni-wc-claims'
file = 'WC_DeEscl_Data_Build_v2.csv'

df = pd.read_csv(f'{bucket}/{file}')
print('Data shape: ', df.shape)

/mnt/miniconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (85,86,87,88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Data shape:  (544529, 100)


In [4]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
# eng_stopwords = set(stopwords.words('english')) 
# extra_stopwords = set(['left', 'right', 'employee', 'emp', 'ee', 'associate', 'iw']) 
# STOPWORDS = set.union(eng_stopwords, extra_stopwords)

def text_prepare(text):
    """
        input: a string
        
        return: cleaned string
    """
    text = text.lower() # lowercase text
    text = re.sub(REPLACE_BY_SPACE_RE, " ", text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = re.sub(BAD_SYMBOLS_RE, "", text) # delete symbols which are in BAD_SYMBOLS_RE from text
#     word_tokens = word_tokenize(text) 
#     word_tokens = [w for w in word_tokens if not w in STOPWORDS] # delete stopwords from text
  
#     stemmer = nltk.stem.WordNetLemmatizer()
#     for i in range(len(word_tokens)):
#         # stemming + spelling correction
        
# #          word_tokens[i] = token.lemma_( word_tokens[i])
#         word_tokens[i] = stemmer.lemmatize(word_tokens[i])
# #         word_tokens[i] = stemmer.lemmatize(spell(word_tokens[i]))
        
#     text = " ".join(word_tokens)
    return text

In [5]:
# process text data
import time
start_time = time.time()

# fill missing notes
df['Accident_Note_Text'] = df['Accident_Note_Text'].fillna('unknown')
df['Injury_Note_Text'] = df['Injury_Note_Text'].fillna('unknown')

df['Accident_Note_Text'] = df['Accident_Note_Text'].apply( text_prepare)
df['Injury_Note_Text'] = df['Injury_Note_Text'].apply(text_prepare)

print("--- %s seconds ---" % (time.time() - start_time))

--- 3.9772567749023438 seconds ---


In [8]:
df['Accident_Note_Text'][0]

'assoicate has been injuring herself on and off now with her restrictions associate said keyboard was about to drop and'

In [20]:
start_time = time.time()

acct_txt_sentmnt = []
inj_txt_sentmnt = []

for i in range(df.shape[0]):
    acct_txt_sentmnt.append(TextBlob(df['Accident_Note_Text'][i]).sentiment.polarity) 
    inj_txt_sentmnt.append(TextBlob(df['Injury_Note_Text'][i]).sentiment.polarity) 

print("--- %s seconds ---" % (time.time() - start_time))

--- 623.4039974212646 seconds ---


In [30]:
df['acct_txt_sentmnt'] = acct_txt_sentmnt
df['inj_txt_sentmnt'] = inj_txt_sentmnt

In [32]:
df[['acct_txt_sentmnt', 'inj_txt_sentmnt', 'deEscInd3000']].groupby(['deEscInd3000']).agg(['count', 'mean'])

acct_txt_sentmnt           inj_txt_sentmnt          
                        count      mean           count      mean
deEscInd3000                                                     
0             147251          -0.002807  147251         -0.001255
1             397278          -0.002491  397278         -0.003138

In [33]:
df[['acct_txt_sentmnt', 'inj_txt_sentmnt', 'deEscInd3000']][df['deEscInd3000'] == 1].describe()

acct_txt_sentmnt  inj_txt_sentmnt  deEscInd3000
count  397278.000000     397278.000000    397278.0    
mean  -0.002491         -0.003138         1.0         
std    0.171766          0.125283         0.0         
min   -1.000000         -1.000000         1.0         
25%   -0.033333         -0.100000         1.0         
50%    0.000000          0.000000         1.0         
75%    0.000000          0.000000         1.0         
max    1.000000          1.000000         1.0

In [34]:
df[['acct_txt_sentmnt', 'inj_txt_sentmnt', 'deEscInd3000']][df['deEscInd3000'] == 0].describe()

acct_txt_sentmnt  inj_txt_sentmnt  deEscInd3000
count  147251.000000     147251.000000    147251.0    
mean  -0.002807         -0.001255         0.0         
std    0.162328          0.130738         0.0         
min   -1.000000         -1.000000         0.0         
25%   -0.047222         -0.100000         0.0         
50%    0.000000          0.000000         0.0         
75%    0.000000          0.000000         0.0         
max    1.000000          0.800000         0.0